In [73]:
import yaml
import copy
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R
pd.options.plotting.backend = 'plotly'
import plotly.graph_objects as go

from mda_core import angle, constants as C, utils, expressions as E
from mda_core.propagation import ephemeris, propagator
from mda_core.propagation.state import State
from mda_core.time import Time
from pathlib import Path
from mda_core import frames
from mda_core.attitude import laws,attitude_providers
from org.hipparchus.geometry.euclidean.threed import RotationConvention, Rotation,RotationOrder


In [74]:
altitude = 565.295
MLTAN = "22:30"
epoch = Time("2024-07-01")
state = State.build_from_sso_params(epoch, altitude, MLTAN)

config_file = Path("C:/Users/Suraj/Documents/mda-core/tests/propagation/config_firefly.yml")
config = yaml.load(config_file.read_text(), yaml.FullLoader)

    # Setting mass
state.set_mass(55.0)
prop = propagator.get_numerical_propagator(state, config, orbit_type="circular")    

2024-07-01 18:32:42.301 | DEBUG    | mda_core.propagation.propagator:_get_integrator:85 - Added Dormand Prince 853 integrator for propagation
2024-07-01 18:32:42.361 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:148 - Added ITRF frame as earth body fixed frame
2024-07-01 18:32:42.363 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:156 - Added gravity harmonics (model : eigen-6s, d x o : 33x33), for the body earth
2024-07-01 18:32:42.363 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:181 - Added third body attraction force contribution from SUN
2024-07-01 18:32:42.365 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:181 - Added third body attraction force contribution from MOON
2024-07-01 18:32:42.367 | DEBUG    | mda_core.propagation.propagator:_get_numerical_forces_from_config:181 - Added third body attraction force contribution from MARS
2024-07-01 18:32:42.36

In [75]:
date=Time("2024-07-01T12:40:00")
sun_axis=[0.2,-0.95,0.35]

In [76]:
from org.orekit.utils import IERSConventions
from org.orekit.frames import FramesFactory, TopocentricFrame
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint
from org.orekit.time import AbsoluteDate
# from org.orekit.orbits import PVCoordinates
from org.orekit.utils import Constants
# from org.orekit.utils import Rotation
import numpy as np

def enu_to_eci_quaternion(lat, lon, date):
    # Set up the frames and ellipsoid
    ITRF = FramesFactory.getITRF(IERSConventions.IERS_2010, True)
    ECI = FramesFactory.getICRF()
    ellipsoid = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                                 Constants.WGS84_EARTH_FLATTENING, ITRF)
    
    # Define the geodetic point
    point = GeodeticPoint(lat,lon, 0.0)
    topo = TopocentricFrame(ellipsoid, point, "topo")

    # Get the transform from ECEF to ECI
    ecef2eci_transform = ITRF.getTransformTo(ECI, date)
    ecef2eci_rotation = ecef2eci_transform.getRotation()

    # Get the transform from ENU to ECEF
    enu2ecef_rotation = topo.getTransformTo(ITRF, date).getRotation()

    # Combine the rotations
    eci2enu_rotation = enu2ecef_rotation.compose(ecef2eci_rotation, RotationConvention.VECTOR_OPERATOR)    
    # Convert to quaternion (w, x, y, z)
    q = eci2enu_rotation.getQ0(), eci2enu_rotation.getQ1(), eci2enu_rotation.getQ2(), eci2enu_rotation.getQ3()
    
    return q

# URSC lat, lon
latitude = angle.deg2rad(12.961826)
longitude = angle.deg2rad(77.654993)
# date = AbsoluteDate.J2000_EPOCH  # Replace with the actual date you need

quaternion = enu_to_eci_quaternion(latitude, longitude, date.get_absolutedate())
print("Quaternion (q_w, q_x, q_y, q_z):", quaternion)


Quaternion (q_w, q_x, q_y, q_z): (-0.5882930864743816, -0.5444713311314177, -0.3022973061703566, -0.515828026249615)


In [77]:
image=date+180

In [78]:
image.get_isostring()

'2024-07-01T12:43:00.000000000'

In [79]:
from org.hipparchus.geometry.euclidean.threed import Vector3D
angle.rad2deg(Vector3D.angle(Vector3D.MINUS_J,utils.list_to_vector3d(sun_axis)))

22.992937732003806

In [80]:
from org.hipparchus.geometry.euclidean.threed import Vector3D
angle.rad2deg(Vector3D.angle(Vector3D.PLUS_I,utils.list_to_vector3d(sun_axis)))

78.82533155821795

In [81]:
prop.getPVCoordinates(date.get_absolutedate(),frames.get_earth_J2000()).getPosition()

<Vector3D: {-925,445.9230522502; -6,141,661.938813525; -3,108,845.646783789}>

In [82]:
prop.getPVCoordinates(date.get_absolutedate(),frames.get_earth_J2000()).getVelocity()

<Vector3D: {-1,614.1832571582; -3,143.6011899576; 6,697.1472504428}>

In [83]:
[laws.get_sun_pointing_with_nadir_phasing(sun_axis,[0,0,1]).getAttitude(prop,date.get_absolutedate(),frames.get_earth_J2000()).getRotation().getQ0(),laws.get_sun_pointing_with_nadir_phasing(sun_axis,[0,0,1]).getAttitude(prop,date.get_absolutedate(),frames.get_earth_J2000()).getRotation().getQ1(),laws.get_sun_pointing_with_nadir_phasing(sun_axis,[0,0,1]).getAttitude(prop,date.get_absolutedate(),frames.get_earth_J2000()).getRotation().getQ2(),laws.get_sun_pointing_with_nadir_phasing(sun_axis,[0,0,1]).getAttitude(prop,date.get_absolutedate(),frames.get_earth_J2000()).getRotation().getQ3()]

[-0.2135184358755946,
 0.558102237170169,
 0.08845473572147133,
 0.7969363400770081]

In [84]:
RPYAngles=laws.get_sun_pointing_with_nadir_phasing(sun_axis,[0,0,1]).getAttitude(prop,date.get_absolutedate(),frames.get_earth_J2000()).getRotation().getAngles(RotationOrder.ZYX, RotationConvention.FRAME_TRANSFORM)
#RPYAngles=laws.get_nadir_pointing().getAttitude(prop,date.get_absolutedate(),frames.get_earth_J2000()).getRotation().getAngles(RotationOrder.ZYX, RotationConvention.FRAME_TRANSFORM)
R = angle.convert_angle(RPYAngles[2], "rad", "deg")
P = angle.convert_angle(RPYAngles[1], "rad", "deg")
Y = angle.convert_angle(RPYAngles[0], "rad", "deg")
R,P,Y

(-15.075274717042728, -68.02042583775888, -139.79833869900898)

In [85]:
rot_body=laws.get_sun_pointing_with_nadir_phasing(sun_axis,[0,0,1]).getAttitude(prop,date.get_absolutedate(),frames.get_earth_J2000()).getRotation()

In [86]:
from org.orekit.frames import LOFType
from org.orekit.attitudes import LofOffset
from org.hipparchus.geometry.euclidean.threed import RotationOrder
lvlh  =LofOffset(frames.get_earth_J2000(),LOFType.LVLH_CCSDS)

In [87]:
LVLH=lvlh.getAttitude(prop,date.get_absolutedate(),frames.get_earth_J2000()).getRotation()
rotation = rot_body.applyTo(LVLH.revert())

RPYAngles = rotation.getAngles(RotationOrder.ZYX, RotationConvention.FRAME_TRANSFORM)

R = angle.convert_angle(RPYAngles[2], "rad", "deg")
P = angle.convert_angle(RPYAngles[1], "rad", "deg")
Y = angle.convert_angle(RPYAngles[0], "rad", "deg")

In [88]:
[rotation.getQ0(),rotation.getQ1(),rotation.getQ2(),rotation.getQ3()]

[-0.8936498693547003,
 0.4382327430087806,
 0.051193327320243696,
 0.08198302992210973]

In [89]:
R,P,Y

(-51.758538022018804, -9.401577015834905, -5.914437105677806)

In [90]:
laws.get_sun_pointing_with_nadir_phasing(sun_axis,[0,0,1]).getAttitude(prop,date.get_absolutedate(),frames.get_earth_J2000()).getSpin()

<Vector3D: {0.000000177; 0.000000071; -0.0000000255}>